In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, log_loss, roc_curve, auc
import timm
from timm import create_model
import torchvision.models as models
import torchio as tio
from torchio import SubjectsLoader
from torchvision import transforms

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define paths based on your directory structure
main_dir = "/content/drive/My Drive/DATASET_MRNET/MRNet-v1.0"
train_path = os.path.join(main_dir, "train")
valid_path = os.path.join(main_dir, "valid")

In [ ]:
# Load labels from CSV files
train_abnormal = pd.read_csv(os.path.join(main_dir, "train-abnormal.csv"), header=None, index_col=0).squeeze("columns").to_dict()
train_acl = pd.read_csv(os.path.join(main_dir, "train-acl.csv"), header=None, index_col=0).squeeze("columns").to_dict()
train_meniscus = pd.read_csv(os.path.join(main_dir, "train-meniscus.csv"), header=None, index_col=0).squeeze("columns").to_dict()

valid_abnormal = pd.read_csv(os.path.join(main_dir, "valid-abnormal.csv"), header=None, index_col=0).squeeze("columns").to_dict()
valid_acl = pd.read_csv(os.path.join(main_dir, "valid-acl.csv"), header=None, index_col=0).squeeze("columns").to_dict()
valid_meniscus = pd.read_csv(os.path.join(main_dir, "valid-meniscus.csv"), header=None, index_col=0).squeeze("columns").to_dict()

In [ ]:
# Function to resize the depth of a scan to a target depth
def resize_depth(scan, target_depth):
    depth_factor = target_depth / scan.shape[0]
    return zoom(scan, (depth_factor, 1, 1), order=1)

In [ ]:
# Function to pad a scan to a target shape
def pad_to_shape(scan, target_shape):
    padded_scan = np.zeros(target_shape, dtype=scan.dtype)
    min_d, min_h, min_w = min(scan.shape[0], target_shape[0]), min(scan.shape[1], target_shape[1]), min(scan.shape[2], target_shape[2])
    padded_scan[:min_d, :min_h, :min_w] = scan[:min_d, :min_h, :min_w]
    return padded_scan

In [ ]:
# Function to load a specific range of MRI data with labels

def load_mri_data(data_type="train", start_idx=0, end_idx=9, target_shape=(48, 256, 256), target_depth=48):
    """
    Loads MRI data from a specified range and resizes/pads each scan to a target shape.
    Parameters:
    - data_type: "train" or "valid"
    - start_idx, end_idx: Range of file indices to load (e.g., 0 to 9 for train, 1130 to 1249 for valid)
    - target_shape: Target shape for each scan after resizing and padding
    - target_depth: Target depth for each scan to ensure consistent depth
    """
    # Set data path and range
    data_path = train_path if data_type == "train" else valid_path
    axial_path, coronal_path, sagittal_path = Path(data_path) / "axial", Path(data_path) / "coronal", Path(data_path) / "sagittal"

    # Select the appropriate labels dictionary based on data type
    abnormal_labels = train_abnormal if data_type == "train" else valid_abnormal
    acl_labels = train_acl if data_type == "train" else valid_acl
    meniscus_labels = train_meniscus if data_type == "train" else valid_meniscus

    # Initialize lists to store data and labels
    mri_data, labels = [], []

    # Load each MRI scan within the specified range
    for i in range(start_idx, end_idx + 1):
        # Generate file name with zero-padded format (e.g., 0000, 0001, ...)
        file_name = f"{i:04}.npy"

        # Load and process each view with resizing and padding
        axial_scan = pad_to_shape(resize_depth(np.load(axial_path / file_name), target_depth), target_shape)
        coronal_scan = pad_to_shape(resize_depth(np.load(coronal_path / file_name), target_depth), target_shape)
        sagittal_scan = pad_to_shape(resize_depth(np.load(sagittal_path / file_name), target_depth), target_shape)

        # Combine the three views into one structure (3, depth, height, width)
        combined_scan = np.stack([axial_scan, coronal_scan, sagittal_scan], axis=0)
        mri_data.append(combined_scan)

        # Retrieve actual labels for the current scan
        abnormal_label = abnormal_labels.get(i, 0)  # Default to 0 if label is missing
        acl_label = acl_labels.get(i, 0)
        meniscus_label = meniscus_labels.get(i, 0)

        # Append the actual labels
        labels.append({"abnormal": abnormal_label, "acl": acl_label, "meniscus": meniscus_label})

    return np.array(mri_data), labels

In [ ]:
# Define the parameters for batch loading with exact final index coverage
start_indices = list(range(0, 1130, 100))
end_indices = [min(start + 99, 1129) for start in start_indices]  # Ensure final batch ends at 1129

# Function to load a batch of data given start and end indices
def load_batch(start, end):
    return load_mri_data(data_type="train", start_idx=start, end_idx=end)

# Initialize lists to store all data and labels
all_data, all_labels = [], []

# Use ThreadPoolExecutor to parallelize data loading for all batches
with ThreadPoolExecutor() as executor:
    # Launch parallel tasks for loading each batch
    future_to_indices = {executor.submit(load_batch, start, end): (start, end) for start, end in zip(start_indices, end_indices)}

    for future in as_completed(future_to_indices):
        data, labels = future.result()
        all_data.append(data)
        all_labels.extend(labels)  # Extend to add lists of labels directly

# Concatenate all data batches into a single array
train_data = np.concatenate(all_data, axis=0)
train_labels = all_labels  # Already extended to combine all label lists

# Check the final shape of training data and labels
print("Final training data shape:", train_data.shape)  # Expected: (1130, 3, 48, 256, 256)
print("Final number of training labels:", len(train_labels))  # Should match the number of samples in train_data


Final training data shape: (1130, 3, 48, 256, 256)
Final number of training labels: 1130


In [ ]:
# Load the validation data from indices 1130 to 1249
valid_data, valid_labels = load_mri_data(data_type="valid", start_idx=1130, end_idx=1249)

# Check data shapes and labels
print("Validation data shape:", valid_data.shape)  # Expected: (120, 3, 48, 256, 256)

Validation data shape: (120, 3, 48, 256, 256)


In [ ]:
# Select the axial view (index 0)
train_data_axial = train_data[:, 0, :, :, :]  # (num_samples, 48, 256, 256)
valid_data_axial = valid_data[:, 0, :, :, :]  # (num_samples, 48, 256, 256)

# Reshape to 2D projections by averaging along the depth dimension
train_data_reshaped = train_data_axial.mean(axis=1)  # Average along depth for projection
valid_data_reshaped = valid_data_axial.mean(axis=1)

# Normalize to [0, 1] range
train_data_normalized = train_data_reshaped / 255.0
valid_data_normalized = valid_data_reshaped / 255.0

# Final shapes for PyTorch: (batch_size, channels, height, width)
# Add channel dimension as the first axis
train_data_final = torch.tensor(train_data_normalized, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
valid_data_final = torch.tensor(valid_data_normalized, dtype=torch.float32).unsqueeze(1)

# Repeat the channel dimension to create 3 channels
train_data_final = train_data_final.repeat(1, 3, 1, 1)
valid_data_final = valid_data_final.repeat(1, 3, 1, 1)

print("Train data shape:", train_data_final.shape)  # Expected: (1130, 3, 256, 256)
print("Validation data shape:", valid_data_final.shape)  # Expected: (120, 3, 256, 256)

Train data shape: torch.Size([1130, 3, 256, 256])
Validation data shape: torch.Size([120, 3, 256, 256])


In [ ]:
# Transform labels into binary matrices for multi-label classification
# Each label dictionary is converted into a list of binary values for each class
train_labels_matrix = [[lbl['abnormal'], lbl['acl'], lbl['meniscus']] for lbl in train_labels]
valid_labels_matrix = [[lbl['abnormal'], lbl['acl'], lbl['meniscus']] for lbl in valid_labels]

# Convert label matrices to PyTorch tensors
# Using float32 because loss functions like BCEWithLogitsLoss expect float inputs
train_labels_encoded = torch.tensor(train_labels_matrix, dtype=torch.float32, device=device)  # Send directly to device
valid_labels_encoded = torch.tensor(valid_labels_matrix, dtype=torch.float32, device=device)  # Send directly to device

# Verify tensor shapes and ensure the data has been prepared correctly
print("Train labels shape:", train_labels_encoded.shape)  # Expected: (1130, 3)
print("Validation labels shape:", valid_labels_encoded.shape)  # Expected: (120, 3)


Train labels shape: torch.Size([1130, 3])
Validation labels shape: torch.Size([120, 3])


In [ ]:
class MRIModel(nn.Module):
    def __init__(self, num_classes=3):
        super(MRIModel, self).__init__()
        self.backbone = timm.create_model('resnet200d', pretrained=True, num_classes=0)
        self.fc1 = nn.Linear(self.backbone.num_features, 512)  # backbone.num_features should be 2048
        self.fc2 = nn.Linear(512, 256)
        self.fc_out = nn.Linear(256, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc_out(x)
        return self.sigmoid(x)


In [ ]:
train_dataset = TensorDataset(train_data_final, train_labels_encoded)
valid_dataset = TensorDataset(valid_data_final, valid_labels_encoded)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

for data, labels in train_loader:
    print(f"Data batch shape: {data.shape}")  # Expected: (batch_size, 3, 256, 256)
    print(f"Labels batch shape: {labels.shape}")  # Expected: (batch_size, 3)
    break


Data batch shape: torch.Size([16, 3, 256, 256])
Labels batch shape: torch.Size([16, 3])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MRIModel(num_classes=3).to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    for data, labels in loader:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)

        # Store predictions and true labels for metrics
        all_preds.append(outputs.detach().cpu())
        all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Compute metrics
    all_preds_binary = (all_preds > 0.5).int()
    accuracy = accuracy_score(all_labels.numpy(), all_preds_binary.numpy())
    precision = precision_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)
    recall = recall_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)

    return running_loss / len(loader.dataset), accuracy, precision, recall



In [ ]:

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data, labels in loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * data.size(0)

            # Store predictions and true labels for metric calculation
            all_preds.append(outputs.cpu())
            all_labels.append(labels.cpu())
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Compute metrics
    all_preds_binary = (all_preds > 0.5).int()
    accuracy = accuracy_score(all_labels.numpy(), all_preds_binary.numpy())
    precision = precision_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)
    recall = recall_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)

    return running_loss / len(loader.dataset), accuracy, precision, recall


In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    # Training
    train_loss, train_accuracy, train_precision, train_recall = train_epoch(
        model, train_loader, criterion, optimizer, device
    )

    # Validation
    val_loss, val_accuracy, val_precision, val_recall = validate_epoch(
        model, valid_loader, criterion, device
    )

    # Print metrics
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}")


Epoch 1/20
Train Loss: 0.5491, Accuracy: 0.3779, Precision: 0.3553, Recall: 0.3401
Validation Loss: 0.6341, Accuracy: 0.1583, Precision: 0.4306, Recall: 0.3590
Epoch 2/20
Train Loss: 0.4359, Accuracy: 0.4292, Precision: 0.6008, Recall: 0.4415
Validation Loss: 0.6245, Accuracy: 0.2250, Precision: 0.4870, Recall: 0.4644
Epoch 3/20
Train Loss: 0.3037, Accuracy: 0.6389, Precision: 0.8191, Recall: 0.6945
Validation Loss: 0.6581, Accuracy: 0.2583, Precision: 0.7466, Recall: 0.6712
Epoch 4/20
Train Loss: 0.1930, Accuracy: 0.7823, Precision: 0.8844, Recall: 0.8451
Validation Loss: 0.6194, Accuracy: 0.3500, Precision: 0.7580, Recall: 0.6391
Epoch 5/20
Train Loss: 0.1308, Accuracy: 0.8425, Precision: 0.9147, Recall: 0.9164
Validation Loss: 0.8839, Accuracy: 0.3667, Precision: 0.7480, Recall: 0.7464
Epoch 6/20
Train Loss: 0.0969, Accuracy: 0.9018, Precision: 0.9525, Recall: 0.9313
Validation Loss: 0.7643, Accuracy: 0.3250, Precision: 0.7335, Recall: 0.6751
Epoch 7/20
Train Loss: 0.0646, Accuracy:

In [ ]:
# Define data transformations with augmentation
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(256, scale=(0.9, 1.0)),
    transforms.ToTensor(),
])

# Update your dataset to apply transformations
class MRIDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data  # Tensor of images
        self.labels = labels  # Tensor of labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    for data, labels in loader:
        # Move data and labels to device here
        data = data.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)

        # Store predictions and true labels for metrics
        all_preds.append(outputs.detach().cpu())
        all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Compute metrics
    all_preds_binary = (all_preds > 0.5).int()
    accuracy = accuracy_score(all_labels.numpy(), all_preds_binary.numpy())
    precision = precision_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)
    recall = recall_score(all_labels.numpy(), all_preds_binary.numpy(), average="macro", zero_division=0)

    return running_loss / len(loader.dataset), accuracy, precision, recall



In [ ]:
train_labels_encoded = torch.tensor(train_labels_matrix, dtype=torch.float32)
valid_labels_encoded = torch.tensor(valid_labels_matrix, dtype=torch.float32)

In [ ]:
# Create datasets
train_dataset = MRIDataset(train_data_final, train_labels_encoded, transform=train_transforms)
valid_dataset = MRIDataset(valid_data_final, valid_labels_encoded)

# Create data loaders with num_workers
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
# Update the model with dropout
class MRIModel(nn.Module):
    def __init__(self, num_classes=3):
        super(MRIModel, self).__init__()
        self.backbone = timm.create_model('resnet50', pretrained=True, num_classes=0)
        self.fc1 = nn.Sequential(
            nn.Linear(self.backbone.num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.fc_out = nn.Linear(256, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.backbone(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc_out(x)
        return self.sigmoid(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, criterion, optimizer, and scheduler
model = MRIModel(num_classes=3).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    # Training
    train_loss, train_accuracy, train_precision, train_recall = train_epoch(
        model, train_loader, criterion, optimizer, device
    )

    # Validation
    val_loss, val_accuracy, val_precision, val_recall = validate_epoch(
        model, valid_loader, criterion, device
    )

    # Step scheduler
    scheduler.step(val_loss)

    # Log the current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Learning Rate: {current_lr:.6f}")
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}\n")


Epoch 1/20
Learning Rate: 0.000100
Train Loss: 0.5975, Accuracy: 0.3655, Precision: 0.4688, Recall: 0.3616
Validation Loss: 0.6817, Accuracy: 0.1667, Precision: 0.2639, Recall: 0.3333

Epoch 2/20
Learning Rate: 0.000100
Train Loss: 0.5371, Accuracy: 0.3779, Precision: 0.3754, Recall: 0.3392
Validation Loss: 0.6494, Accuracy: 0.1667, Precision: 0.2639, Recall: 0.3333

Epoch 3/20
Learning Rate: 0.000100
Train Loss: 0.5276, Accuracy: 0.3823, Precision: 0.4249, Recall: 0.3392
Validation Loss: 0.6932, Accuracy: 0.1667, Precision: 0.2639, Recall: 0.3333

Epoch 4/20
Learning Rate: 0.000100
Train Loss: 0.5281, Accuracy: 0.3823, Precision: 0.4152, Recall: 0.3392
Validation Loss: 0.6522, Accuracy: 0.1667, Precision: 0.2639, Recall: 0.3333

Epoch 5/20
Learning Rate: 0.000100
Train Loss: 0.5191, Accuracy: 0.3832, Precision: 0.4499, Recall: 0.3442
Validation Loss: 0.6667, Accuracy: 0.1667, Precision: 0.2639, Recall: 0.3333

Epoch 6/20
Learning Rate: 0.000100
Train Loss: 0.4978, Accuracy: 0.4035, Pr

In [ ]:
# Prepare subjects for training data
train_subjects = []
for i in range(len(train_data)):
    image_tensor = train_data[i]  # Shape: (C, D, H, W)
    label = train_labels[i]
    image = tio.ScalarImage(tensor=image_tensor)
    subject = tio.Subject(
    image=image,
    abnormal=label['abnormal'],
    acl=label['acl'],
    meniscus=label['meniscus']
    )
    train_subjects.append(subject)

# Prepare subjects for validation data
valid_subjects = []
for i in range(len(valid_data)):
    image_tensor = valid_data[i]
    label = valid_labels[i]
    image = tio.ScalarImage(tensor=image_tensor)
    subject = tio.Subject(
    image=image,
    abnormal=label['abnormal'],
    acl=label['acl'],
    meniscus=label['meniscus']
    )
    valid_subjects.append(subject)

# Create datasets using tio.SubjectsDataset
train_dataset = tio.SubjectsDataset(train_subjects, transform=train_transforms)
valid_dataset = tio.SubjectsDataset(valid_subjects, transform=valid_transforms)

In [ ]:
# Define transformations using TorchIO
train_transforms = tio.Compose([
    tio.RandomFlip(axes=('LR', 'AP', 'SI'), flip_probability=0.5, include=('image',)),
    tio.RandomAffine(scales=(0.9, 1.1), degrees=10, include=('image',)),
    tio.ZNormalization(include=('image',)),
])

valid_transforms = tio.Compose([
    tio.ZNormalization(include=('image',)),
])

In [ ]:
train_loader = SubjectsLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4)
valid_loader = SubjectsLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4)

In [ ]:
print(type(train_dataset[0]))

<class 'torchio.data.subject.Subject'>


In [ ]:
class MRI3DResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(MRI3DResNet, self).__init__()
        # Load a pretrained 3D ResNet
        self.backbone = models.video.r3d_18(pretrained=True)
        # Modify the first convolutional layer to accept 3 input channels
        self.backbone.stem[0] = nn.Conv3d(
            in_channels=3,  # Change to 3 channels
            out_channels=64,
            kernel_size=(3, 7, 7),
            stride=(1, 2, 2),
            padding=(1, 3, 3),
            bias=False
        )
        # Replace the fully connected layer
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)
        # No activation here because we'll use BCEWithLogitsLoss

    def forward(self, x):
        x = self.backbone(x)
        return x

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MRI3DResNet(num_classes=3).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
print(type(train_subjects[0]))

<class 'torchio.data.subject.Subject'>


In [ ]:
import torch
print(torch.__version__)

2.5.1+cu121


In [ ]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Parameter {name} does not require gradient")

In [ ]:
num_epochs = 20
warnings.filterwarnings("ignore", category=UserWarning, module="torchio")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []
    for batch in train_loader:
        data = batch['image'][tio.DATA].to(device)
        labels = torch.tensor(
            [batch['abnormal'], batch['acl'], batch['meniscus']],
            dtype=torch.float32
        ).T.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)

        # Collect predictions and labels for metrics
        outputs = torch.sigmoid(outputs)
        preds = (outputs > 0.5).float()
        all_train_preds.append(preds.cpu())
        all_train_labels.append(labels.cpu())

    # Compute training metrics
    all_train_preds = torch.cat(all_train_preds)
    all_train_labels = torch.cat(all_train_labels)
    train_accuracy = accuracy_score(all_train_labels.numpy(), all_train_preds.numpy())
    train_precision = precision_score(all_train_labels.numpy(), all_train_preds.numpy(), average='weighted', zero_division=0)
    train_recall = recall_score(all_train_labels.numpy(), all_train_preds.numpy(), average='weighted', zero_division=0)
    epoch_loss = running_loss / len(train_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Training Loss: {epoch_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")

    model.eval()
    val_running_loss = 0.0
    all_val_preds = []
    all_val_labels = []
    with torch.no_grad():
        for batch in valid_loader:
            data = batch['image'][tio.DATA].to(device)
            labels = torch.tensor(
                [batch['abnormal'], batch['acl'], batch['meniscus']],
                dtype=torch.float32
            ).T.to(device)

            outputs = model(data)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * data.size(0)

            # Collect predictions and labels for metrics
            outputs = torch.sigmoid(outputs)
            preds = (outputs > 0.5).float()
            all_val_preds.append(preds.cpu())
            all_val_labels.append(labels.cpu())

    # Compute validation metrics
    all_val_preds = torch.cat(all_val_preds)
    all_val_labels = torch.cat(all_val_labels)
    val_accuracy = accuracy_score(all_val_labels.numpy(), all_val_preds.numpy())
    val_precision = precision_score(all_val_labels.numpy(), all_val_preds.numpy(), average='weighted', zero_division=0)
    val_recall = recall_score(all_val_labels.numpy(), all_val_preds.numpy(), average='weighted', zero_division=0)
    val_loss = val_running_loss / len(valid_dataset)

    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}\n")


Epoch 1/20
Training Loss: 0.7019, Accuracy: 0.0000, Precision: 0.0919, Recall: 0.2615
Validation Loss: 0.8315, Accuracy: 0.0000, Precision: 0.4497, Recall: 0.2836



KeyboardInterrupt: 